# Teste brreg API
- <https://data.brreg.no/enhetsregisteret/oppslag/enheter?kommunenummer=1103,1108&fraAntallAnsatte=10&tilAntallAnsatte=50000&sort=navn.norwegian,asc>

In [1]:
import requests
import pandas as pd
from brreg.enhetsregisteret import Client



In [23]:
df = pd.read_json('enheter_sandnes_stavanger.json')
df['naeringskode_1'] = df['naeringskode1'].apply(lambda x: x['kode'])
df['naeringsbeskrivelse_1'] = df['naeringskode1'].apply(lambda x: x['beskrivelse'])
df['organisasjonsform'] = df['organisasjonsform'].apply(lambda x: x['beskrivelse'])
#need to handle nan value for  naeringskode2 and 3
# Function to extract 'kode' from dictionary or return None for NaN
def extract_kode(x):
    if isinstance(x, dict):
        return x['kode']
    else:
        return None

# Function to extract 'beskrivelse' from dictionary or return None for NaN
def extract_beskrivelse(x):
    if isinstance(x, dict):
        return x['beskrivelse']
    else:
        return None
df['naeringskode_2'] = df['naeringskode2'].apply(extract_kode)
df['naeringsbeskrivelse_2'] = df['naeringskode2'].apply(extract_beskrivelse)
df['naeringskode_3'] = df['naeringskode3'].apply(extract_kode)
df['naeringsbeskrivelse_3'] = df['naeringskode3'].apply(extract_beskrivelse)
df['kommune'] = df['forretningsadresse'].apply(lambda x: x['kommune'])
df['forretningsadresse'] = df['forretningsadresse'].apply(lambda x: f"{', '.join(x['adresse'])}, {x['postnummer']} {x['poststed']}")
#change format of overordnet_enhet to text and remove decimal from float representation
df['overordnetEnhet'] = df['overordnetEnhet'].astype(str).apply(lambda x: x.rstrip('.0'))
df['organisasjonsnummer'] = df['organisasjonsnummer'].astype(str)

In [27]:
# Convert list to string
df['vedtektsfestetFormaal'] = df['vedtektsfestetFormaal'].apply(lambda x: ', '.join(map(str, x)) if isinstance(x, list) else "NaN")
df['aktivitet'] = df['aktivitet'].apply(lambda x: ', '.join(map(str, x)) if isinstance(x, list) else "NaN")
# Convert to integer
df['sisteInnsendteAarsregnskap'] = pd.to_numeric(df['sisteInnsendteAarsregnskap'], downcast='integer').astype('Int64')

## Select columns for companies to store in csv

In [10]:
df.columns

Index(['organisasjonsnummer', 'navn', 'organisasjonsform',
       'registreringsdatoEnhetsregisteret', 'registrertIMvaregisteret',
       'naeringskode1', 'antallAnsatte', 'harRegistrertAntallAnsatte',
       'forretningsadresse', 'stiftelsesdato', 'institusjonellSektorkode',
       'registrertIForetaksregisteret', 'registrertIStiftelsesregisteret',
       'registrertIFrivillighetsregisteret', 'konkurs', 'underAvvikling',
       'underTvangsavviklingEllerTvangsopplosning', 'maalform', 'vedtektsdato',
       'vedtektsfestetFormaal', 'aktivitet', 'links',
       'sisteInnsendteAarsregnskap', 'frivilligMvaRegistrertBeskrivelser',
       'naeringskode2', 'naeringskode3', 'postadresse', 'hjemmeside',
       'hjelpeenhetskode', 'overordnetEnhet', 'naeringskode_1',
       'naeringsbeskrivelse_1', 'naeringskode_2', 'naeringsbeskrivelse_2',
       'naeringskode_3', 'naeringsbeskrivelse_3', 'kommune'],
      dtype='object')

## Create data for company codes and prepare for relationships

In [20]:
business_1 = df[['naeringskode_1', 'naeringsbeskrivelse_1']].drop_duplicates()
business_2 = df[['naeringskode_2', 'naeringsbeskrivelse_2']].drop_duplicates()
business_3 = df[['naeringskode_3', 'naeringsbeskrivelse_3']].drop_duplicates()
business_2.columns = business_1.columns
business_3.columns = business_1.columns
business = pd.concat([business_1, business_2, business_3])

# Check for and remove duplicates
business = business.drop_duplicates()
# Remove _1 from column names   
business.columns = business.columns.str.replace('_1', '')
business.to_csv('csv_files/business_codes.csv', index=False)

# Convert list to string
df['vedtektsfestetFormaal'] = df['vedtektsfestetFormaal'].apply(lambda x: ', '.join(map(str, x)) if isinstance(x, list) else "NaN")
df['aktivitet'] = df['aktivitet'].apply(lambda x: ', '.join(map(str, x)) if isinstance(x, list) else "NaN")
# Convert to integer
df['sisteInnsendteAarsregnskap'] = pd.to_numeric(df['sisteInnsendteAarsregnskap'], downcast='integer').astype('Int64')

## Company relationships

In [100]:
df[['organisasjonsnummer','naeringskode_1']].to_csv('csv_files/rel_business_1.csv', index=False)
df[df['naeringskode_2'].notna()][['organisasjonsnummer','naeringskode_2']].to_csv('csv_files/rel_business_2.csv', index=False)
df[df['naeringskode_3'].notna()][['organisasjonsnummer','naeringskode_3']].to_csv('csv_files/rel_business_3.csv', index=False)

In [106]:
df[-['naeringskode1']]

TypeError: bad operand type for unary -: 'list'

# Look at overordnet enhet

In [7]:
df[df['overordnetEnhet'] != "nan"]['overordnetEnhet']

654     991012133
719      97716163
813     971045698
814     971045698
815     971045698
816     971045698
817     971045698
818     971045698
967     983887406
968     972417858
983     964965226
984     964965226
985     964965226
986     964965226
988     964965226
998     964965226
1097    872417842
1431    983887457
1518    964965226
1541    915429785
1543    971045698
Name: overordnetEnhet, dtype: object

# Store company data as csv

In [29]:
df[['organisasjonsnummer', 'navn', 'organisasjonsform',
       'registreringsdatoEnhetsregisteret',  'antallAnsatte', 
       'forretningsadresse', 'stiftelsesdato', 'vedtektsdato',
       'vedtektsfestetFormaal', 'aktivitet', 'links',
       'sisteInnsendteAarsregnskap', 'kommune','overordnetEnhet']].to_csv('csv_files/enheter.csv', index=False)

,organisasjonsnummer,navn,organisasjonsform,registreringsdatoEnhetsregisteret,antallAnsatte,forretningsadresse,stiftelsesdato,vedtektsdato,vedtektsfestetFormaal,aktivitet,links,sisteInnsendteAarsregnskap,kommune,overordnetEnhet
0,931648160,24/7 BYGGRENT AS,Aksjeselskap,2023-06-27,10,"Svend Foyns gate 48, 4016 STAVANGER",2023-05-11,2023-05-11,"Rengjøringsbyrå, utvendig og innvendig rengjør...","Rengjøring av bygninger utvendig, vinduer og l...",[],<NA>,STAVANGER,nan
1,925178853,A. IDSØE STAVANGER AS,Aksjeselskap,2020-06-10,32,"Verksgata 3, 4013 STAVANGER",2020-04-30,2021-02-01,Kjøp og salg samt produksjon av næringsmidler ...,Produksjon av kjøtt- og fjørfevarer samt kjøp ...,[],2022,STAVANGER,nan
2,914807077,A/S Norske Shell,Aksjeselskap,1995-03-12,464,"Løkkeveien 103, 4007 STAVANGER",1912-10-14,2014-05-23,"- Lete etter og utvinne petroleum,, , - raffi...","- Lete etter og utvinne petroleum,, , - raffi...",[],2022,STAVANGER,nan
3,890046452,A2 HOLDING AS,Aksjeselskap,2006-08-10,58,"c/o Ragnhild Eielsen Wiig, Mosestien 8B, 4018 ...",2006-06-29,2011-05-13,Eie og drive barnehage i Stavanger. Selskapet ...,Bygging og drift av barnehage.,[],2022,STAVANGER,nan
4,928285553,A3 TRANSPORT AS,Aksjeselskap,2021-12-06,13,"Nikkelveien 8, 4313 SANDNES",2021-11-24,2022-08-12,Transport og spedisjonsvirksomhet og hva derme...,Transport og spedisjonsvirksomhet og hva derme...,[],2022,SANDNES,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1667,999236022,NABOEN AS,Aksjeselskap,2012-12-06,14,"Maskinveien 25, 4033 STAVANGER",2012-10-26,2022-04-28,Investering i andre selskaper.,Investering i andre selskaper.,[],2022,STAVANGER,nan
1668,999530117,TRYGG EIVIND SKJELDE AS,Aksjeselskap,2013-01-29,17,"Haugesundsgata 43, 4014 STAVANGER",2012-12-17,2012-12-17,"Selv, gjennom hel- eller deleide selskaper ell...",Butikkhandel med dagligvarer.,[],2022,STAVANGER,nan
1669,919415630,DESERT CONTROL AS,Aksjeselskap,2017-09-07,12,"FOMO Works, Grenseveien 21, 4313 SANDNES",2017-06-27,2023-11-17,"Utvikle, produsere, markedsføre og selge produ...","utvikle, produsere, markedsføre og selge utsty...",[],2022,SANDNES,nan
1670,918324909,NMS DRIFT AS,Aksjeselskap,2017-01-06,87,"Misjonsmarka 12, 4024 STAVANGER",2016-12-19,2018-12-21,Realisere formålene til Det Norske Misjonssels...,Drifte NMS sine eiendommer.,[],2022,STAVANGER,nan


In [25]:
type(df["vedtektsfestetFormaal"].iloc[0])

list

In [9]:
# /enhetsregisteret/api/enheter/{orgnr}
# Define the API endpoint URL
url = "https://data.brreg.no/enhetsregisteret/oppslag/enheter/974760673/roller"

# Define query parameters
params = {"orgnr": 974760673}

# Make an HTTP GET request to the API
response = requests.get(url, params=params)

In [10]:
# Define the API endpoint URL
orgnr = '915501680'
url = f'https://data.brreg.no/enhetsregisteret/api/enheter/{orgnr}/roller'

# Define headers for the request
headers = {
    'Accept': 'application/json'
}

# Make an HTTP GET request to the API with the specified headers
response = requests.get(url, headers=headers)
data = response.json()

In [12]:
data.to_json('roles.json')

AttributeError: 'dict' object has no attribute 'to_json'

In [13]:
# Initialize empty lists to store data
names = []
roles = []
dates_of_birth = []

# Loop through the JSON response data and extract the desired information
for role_group in data['rollegrupper']:
    role_group_description = role_group['type']['beskrivelse']
    for role in role_group['roller']:
        person = role.get('person', {})  # Check if 'person' key exists, and if not, use an empty dictionary
        name = role['enhet']['navn'][0]
        # Check if "date_of_birth" is available (for a person)
        if 'person' in data:
            date_of_birth = data['person'].get('fodselsdato', '')
        else:
            date_of_birth = ''
        # Append data to lists
        names.append(name)
        roles.append(role_group_description)
        dates_of_birth.append(date_of_birth)

# Create a Pandas DataFrame
df = pd.DataFrame({
    'name': names,
    'role': roles,
    'date_of_birth': dates_of_birth
})

# Display the DataFrame
print(df)

KeyError: 'enhet'

In [14]:
person

{'fodselsdato': '1980-02-16',
 'navn': {'fornavn': 'Andreas',
  'mellomnavn': 'Egge',
  'etternavn': 'Thorsheim'},
 'erDoed': False}

In [28]:
for role_group in data['rollegrupper']:
    role_group_description = role_group['type']['beskrivelse']
    for role in role_group['roller']:
        person = role.get('person', {}) 

In [38]:
role['enhet']['navn'][0]

'AIDER ACCOUNTING AS'

In [10]:

client = Client()
enhet = client.get_enhet('915501680')

In [11]:
enhet

Enhet(organisasjonsnummer='915501680', navn='OTOVO ASA', organisasjonsform=Organisasjonsform(kode='ASA', beskrivelse='Allmennaksjeselskap', utgaatt=None), hjemmeside='www.otovo.no', postadresse=None, registreringsdato_enhetsregisteret=datetime.date(2015, 7, 10), registrert_i_mvaregisteret=True, frivillig_mva_registrert_beskrivelser=[], naeringskode1=Naeringskode(kode='46.180', beskrivelse='Agenturhandel med spesialisert vareutvalg ellers'), naeringskode2=None, naeringskode3=None, hjelpeenhetskode=None, antall_ansatte=91, har_registrert_antall_ansatte=True, overordnet_enhet=None, forretningsadresse=Adresse(adresse=['Torggata 7'], postnummer='0181', poststed='OSLO', kommunenummer='0301', kommune='OSLO', landkode='NO', land='Norge'), stiftelsesdato=datetime.date(2015, 6, 1), institusjonell_sektorkode=InstitusjonellSektorkode(kode='2100', beskrivelse='Private aksjeselskaper mv.'), registrert_i_foretaksregisteret=True, registrert_i_stiftelsesregisteret=False, registrert_i_frivillighetsregis

# Unique constraints
```cypher
CREATE CONSTRAINT organisasjonsnummer_unique IF NOT EXISTS FOR (c:Company) REQUIRE c.organisasjonsnummer IS UNIQUE

```

In [ ]:
##